# Computer Vision Lab – Convolution on Images

This Jupyter notebook demonstrates **2D convolution operations on images** using custom kernels.

Topics covered:
- What convolution is in the context of images
- Implementing convolution manually with NumPy
- Using OpenCV's `filter2D` for convolution
- Applying common kernels:
  - Box blur / averaging filter
  - Gaussian-like blur
  - Sharpening
  - Edge detection (Sobel, Laplacian)
  - Emboss
  - Motion blur

**Note:** Before running, place an image file (e.g., `input.jpg`) in the same folder as this notebook, or update the image path accordingly.

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

# For inline display in Jupyter
%matplotlib inline

## Helper Functions

In [ ]:
def show_bgr(img, title="Image"):
    """Display a BGR OpenCV image with matplotlib in RGB order."""
    if img is None:
        raise ValueError("Image is None. Check the path.")
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    plt.figure(figsize=(5, 5))
    plt.imshow(img_rgb)
    plt.title(title)
    plt.axis('off')
    plt.show()

def show_gray(img, title="Image"):
    """Display a grayscale image."""
    if img is None:
        raise ValueError("Image is None. Check the path.")
    plt.figure(figsize=(5, 5))
    plt.imshow(img, cmap='gray')
    plt.title(title)
    plt.axis('off')
    plt.show()

## Load Input Image

In [ ]:
# Change this path to your image file if needed
image_path = 'input.jpg'

img = cv2.imread(image_path)
if img is None:
    raise FileNotFoundError(f"Could not read image from {image_path}. Please check the path.")

gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

show_bgr(img, "Original Color Image")
show_gray(gray, "Original Grayscale Image")

## Manual Implementation of 2D Convolution

Below is a **from-scratch implementation** of 2D convolution on a single-channel (grayscale) image.

Steps:
1. Pad the image so that the kernel can slide over all pixels.
2. For each output pixel, take the corresponding neighborhood.
3. Multiply element-wise with the kernel and sum the result.

In [ ]:
def convolve2d(gray_img, kernel):
    """Manual 2D convolution (single channel) with zero-padding.

    gray_img: 2D NumPy array
    kernel: 2D NumPy array
    """
    if len(gray_img.shape) != 2:
        raise ValueError("Input image must be grayscale (2D array).")

    k_h, k_w = kernel.shape
    pad_h = k_h // 2
    pad_w = k_w // 2

    # Flip kernel for convolution
    kernel_flipped = np.flipud(np.fliplr(kernel))

    # Pad image
    padded = np.pad(gray_img, ((pad_h, pad_h), (pad_w, pad_w)), mode='constant', constant_values=0)

    out = np.zeros_like(gray_img, dtype=np.float32)

    for i in range(gray_img.shape[0]):
        for j in range(gray_img.shape[1]):
            region = padded[i:i + k_h, j:j + k_w]
            out[i, j] = np.sum(region * kernel_flipped)

    # Clip to valid range
    out = np.clip(out, 0, 255).astype(np.uint8)
    return out

# Example: simple 3x3 averaging kernel
avg_kernel = np.ones((3, 3), np.float32) / 9.0
manual_blur = convolve2d(gray, avg_kernel)
show_gray(manual_blur, "Manual Convolution: 3x3 Averaging Filter")

## Convolution with OpenCV `filter2D`

OpenCV provides a convenient function `cv2.filter2D` that performs convolution with a given kernel.

In [ ]:
def apply_kernel_cv(img, kernel, title_prefix=""):
    """Apply a kernel to a BGR image and show result."""
    filtered = cv2.filter2D(img, ddepth=-1, kernel=kernel)
    show_bgr(filtered, f"{title_prefix}")
    return filtered

# Test with the same averaging kernel on color image
box_kernel_3x3 = np.ones((3, 3), np.float32) / 9.0
_ = apply_kernel_cv(img, box_kernel_3x3, "Box Blur 3x3 (Color)")

## Smoothing Filters (Blurring)

In [ ]:
# 1. Box blur (3x3 and 5x5)
box_kernel_5x5 = np.ones((5, 5), np.float32) / 25.0
_ = apply_kernel_cv(img, box_kernel_3x3, "Box Blur 3x3")
_ = apply_kernel_cv(img, box_kernel_5x5, "Box Blur 5x5")

# 2. Gaussian-like kernel (3x3)
gaussian_kernel_3x3 = (1/16.0) * np.array([[1, 2, 1],
                                           [2, 4, 2],
                                           [1, 2, 1]], dtype=np.float32)
_ = apply_kernel_cv(img, gaussian_kernel_3x3, "Gaussian-like Blur 3x3")

## Sharpening Filter

In [ ]:
sharpen_kernel = np.array([[0, -1, 0],
                           [-1, 5, -1],
                           [0, -1, 0]], dtype=np.float32)

_ = apply_kernel_cv(img, sharpen_kernel, "Sharpened Image")

## Edge Detection Filters (Sobel, Laplacian)

In [ ]:
# Sobel - Horizontal and Vertical (in kernel form)
sobel_x = np.array([[-1, 0, 1],
                    [-2, 0, 2],
                    [-1, 0, 1]], dtype=np.float32)

sobel_y = np.array([[-1, -2, -1],
                    [ 0,  0,  0],
                    [ 1,  2,  1]], dtype=np.float32)

edges_x = convolve2d(gray, sobel_x)
edges_y = convolve2d(gray, sobel_y)

show_gray(edges_x, "Sobel X (Horizontal Edges)")
show_gray(edges_y, "Sobel Y (Vertical Edges)")

# Combined magnitude approximation
edges_combined = cv2.addWeighted(edges_x, 0.5, edges_y, 0.5, 0)
show_gray(edges_combined, "Combined Sobel Edges")

# Laplacian kernel
laplacian_kernel = np.array([[0, 1, 0],
                             [1, -4, 1],
                             [0, 1, 0]], dtype=np.float32)

laplacian_edges = convolve2d(gray, laplacian_kernel)
show_gray(laplacian_edges, "Laplacian Edges")

## Emboss Filter

In [ ]:
emboss_kernel = np.array([[-2, -1, 0],
                          [-1,  1, 1],
                          [ 0,  1, 2]], dtype=np.float32)

embossed = apply_kernel_cv(img, emboss_kernel, "Emboss Effect")

## Motion Blur Filter

In [ ]:
size = 9
motion_kernel = np.zeros((size, size), dtype=np.float32)
for i in range(size):
    motion_kernel[i, i] = 1.0
motion_kernel /= size

motion_blurred = apply_kernel_cv(img, motion_kernel, "Motion Blur (Diagonal)")

## Summary

In this notebook, we:
- Implemented **2D convolution manually** using NumPy.
- Used **OpenCV's `filter2D`** to apply kernels on images.
- Applied several common convolution kernels:
  - Box blur / averaging
  - Gaussian-like smoothing
  - Sharpening
  - Edge detection (Sobel, Laplacian)
  - Emboss
  - Motion blur

You can experiment further by:
- Designing your own kernels
- Changing kernel sizes
- Applying these operations to different images
- Combining filters (e.g., blur → sharpen → edge detection).